## First Layer

In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

import datetime
import time
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
from datetime import date
date_today = date.today()

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')
driver.set_window_position(0, 0)
driver.set_window_size(1248, 768)

driver.get("https://www.gap.com/browse/category.do?cid=11900&nav=meganav%3AMen%3ANew%20%26%20Ready%20to%20Gift%3ANew%20Arrivals#pageId=0&department=75")
# https://www.gap.com/browse/category.do?cid=8792#pageId=0&department=136&mlink=5058,19400081,HP_VCN_2_W_G31726_IMAGE
time.sleep(1.5)

X_button = driver.find_element_by_class_name('css-1qosac6')
X_button.click()

last_height = driver.execute_script("return document.body.scrollHeight")
counter = 1
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollBy(0, 768);")

    # Wait to load page
    time.sleep(.3)

    # Calculate new scroll height and compare with last scroll height
    new_height = 768 * counter
    if new_height >= last_height:
        break
    
    counter += 1

soup = BeautifulSoup(driver.page_source,'html.parser')


In [55]:
product_name = []
original_price = []
sale_price = []
product_url = []
image_url = []
brand = []
product_type = []
timestamp = []

for i in soup.find_all('div',{'class':'product-card'}):
    if i.find('div',{'class':'product-card__name'}) != None:
        product_name.append((i.find('div',{'class':'product-card__name'})).text.strip())
    else:
        product_name.append(None)
    if i.find('span',{'class':'product-price__no-strike'}) != None:
        original_price.append((i.find('span',{'class':'product-price__no-strike'})).text.strip())
    else:
        original_price.append(None)
    if i.find('div',{'class':'product-price__highlight'}) != None:
        sale_price.append((i.find('div',{'class':'product-price__highlight'})).text.strip().replace("Now ",""))
    elif i.find('div',{'class':'product-card-price'}) != None:
        sale_price.append((i.find('div',{'class':'product-card-price'})).text.strip())
    else:
        sale_price.append(None)
    product_url.append(i.find('a',{'class':'css-0'}).get('href'))
    if (i.find('div',{'class':'product-card__image-wrapper'}).find('img',{'class':'product-card__image'})) !=None:
        image_url.append((i.find('div',{'class':'product-card__image-wrapper'}).find('img',{'class':'product-card__image'})).get('src'))
    else:
        image_url.append(None)
    brand.append("gap")
    product_type.append("Men's Clothing")
    timestamp.append(date_today)

In [56]:
column_names = ["product_name", "original_price", "sale_price",'product_link', 'image_link']
df = pd.DataFrame(columns = column_names)

df['product_name'] = product_name
df['original_price'] = original_price
df['sale_price'] = sale_price
df['image_link'] = image_url
df['product_link'] = product_url

In [57]:
df

,product_name,original_price,sale_price,product_link,image_link
0,Vintage Soft Hoodie,$54.95,$49.00,https://www.gap.com/browse/product.do?pid=6187...,https://www4.assets-gap.com/webcontent/0019/69...
1,Vintage Soft Hoodie,$54.95,$49.00,https://www.gap.com/browse/product.do?pid=6187...,https://www3.assets-gap.com/webcontent/0019/52...
2,Vintage Soft Hoodie,$54.95,$49.00,https://www.gap.com/browse/product.do?pid=6187...,https://www2.assets-gap.com/webcontent/0019/69...
3,Vintage Soft Sweatshirt,$49.95,$44.00,https://www.gap.com/browse/product.do?pid=6187...,https://www4.assets-gap.com/webcontent/0019/49...
4,Vintage Soft Sweatshirt,$49.95,$44.00,https://www.gap.com/browse/product.do?pid=6187...,https://www3.assets-gap.com/webcontent/0019/49...
...,...,...,...,...,...
278,Adult Face Mask (3-Pack),$15.00,$5.00,https://www.gap.com/browse/product.do?pid=6688...,https://www1.assets-gap.com/webcontent/0018/87...
279,Adult Face Mask (3-Pack),$15.00,$5.00,https://www.gap.com/browse/product.do?pid=6688...,https://www2.assets-gap.com/webcontent/0018/90...
280,Adult Face Mask (3-Pack),$15.00,$5.00,https://www.gap.com/browse/product.do?pid=6688...,https://www1.assets-gap.com/webcontent/0018/88...
281,Adult Face Mask (3-Pack),$15.00,$5.00,https://www.gap.com/browse/product.do?pid=6688...,https://www1.assets-gap.com/webcontent/0018/87...


## Second Layer

In [58]:
product_description = []
material = []
picture_name = []
color = []
counter = 1
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')
driver.set_window_position(0, 0)
driver.set_window_size(700, 768)

for link in product_url:
    driver.get(link)
    time.sleep(1.5)
    
    # in the case of not in stock
    soup = BeautifulSoup(driver.page_source,'html.parser')
    if soup.find('h1',{'class':'product-title__text'}) != None:
        #X_button = driver.find_element_by_class_name('css-1qosac6')
        #X_button.click()
        #time.sleep(.3)
        #promo_button = driver.find_element_by_css_selector('button[aria-label="Close Promo Drawer"]')
        #detail_button.click()
        #time.sleep(.1)
        #driver.execute_script("window.scrollTo(0, 1720);")
        #time.sleep(.5)
        #detail_button = driver.find_element_by_xpath('/html/body/div[2]/div/div[1]/div/div[1]/div[2]/div/div/div/div[1]/div/div[2]/button[2]')
        #detail_button.click()
        #time.sleep(.5)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        item_color = []
        if soup.find('div',{'class':'swatch-group sds_clearfix swatch-group--color'}).find('img') != None:
            for color_i in soup.find('div',{'class':'swatch-group sds_clearfix swatch-group--color'}).find_all('img'):
                item_color.append(color_i.get("alt"))
            color.append(item_color)
        else:
            color.append(None)

        material.append(None)

        item_description = []
        if soup.find_all('li',{'class':'product-information-item__list-item'}) != None:
            for i in soup.find_all('li',{'class':'product-information-item__list-item'}):
                item_description.append(i.text.strip())
            product_description.append(item_description)
        else:
            product_description.append(None)
    else:
        color.append(None)
        material.append(None)
        product_description.append(None)
    

    picture_name.append('gap-men_' + str(counter) + '.jpg') # change variable
    counter += 1
driver.close()

In [59]:
for i,v in enumerate(product_description):
    if v != None:
        product_description[i] = v[0:3]
        item_material = []
        for j in v:
            if "%" in j:
                item_material.append(j)
        material[i] = item_material

In [60]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = product_name
second_pass.product_description = product_description
second_pass.price = original_price
second_pass.sales_price = sale_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = image_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,gap,Men's Clothing,Vintage Soft Hoodie,"[Straight silhouette with a relaxed fit., Pull...",$54.95,$49.00,"[admiral blue, neon coral flame, french almond...","[77% Cotton, 14% Polyester, 9% Recycled Polyes...",https://www.gap.com/browse/product.do?pid=6187...,https://www4.assets-gap.com/webcontent/0019/69...,2020-12-19,gap-men_1.jpg
1,gap,Men's Clothing,Vintage Soft Hoodie,"[Straight silhouette with a relaxed fit., Pull...",$54.95,$49.00,"[admiral blue, neon coral flame, french almond...","[77% Cotton, 14% Polyester, 9% Recycled Polyes...",https://www.gap.com/browse/product.do?pid=6187...,https://www3.assets-gap.com/webcontent/0019/52...,2020-12-19,gap-men_2.jpg
2,gap,Men's Clothing,Vintage Soft Hoodie,"[Straight silhouette with a relaxed fit., Pull...",$54.95,$49.00,"[admiral blue, neon coral flame, french almond...","[77% Cotton, 14% Polyester, 9% Recycled Polyes...",https://www.gap.com/browse/product.do?pid=6187...,https://www2.assets-gap.com/webcontent/0019/69...,2020-12-19,gap-men_3.jpg
3,gap,Men's Clothing,Vintage Soft Sweatshirt,"[Straight silhouette with a relaxed fit., Pull...",$49.95,$44.00,"[yellow jacket, rust, light blue shadow, potpo...","[77% Cotton, 14% Polyester, 9% Recycled Polyes...",https://www.gap.com/browse/product.do?pid=6187...,https://www4.assets-gap.com/webcontent/0019/49...,2020-12-19,gap-men_4.jpg
4,gap,Men's Clothing,Vintage Soft Sweatshirt,"[Straight silhouette with a relaxed fit., Pull...",$49.95,$44.00,"[yellow jacket, rust, light blue shadow, potpo...","[77% Cotton, 14% Polyester, 9% Recycled Polyes...",https://www.gap.com/browse/product.do?pid=6187...,https://www3.assets-gap.com/webcontent/0019/49...,2020-12-19,gap-men_5.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
278,gap,Men's Clothing,Adult Face Mask (3-Pack),"[7.5"" x 4"", Made of triple-layer cotton., Mixe...",$15.00,$5.00,"[true black, multi black green blue, navy blue...",[100% Cotton Printed Poplin.],https://www.gap.com/browse/product.do?pid=6688...,https://www1.assets-gap.com/webcontent/0018/87...,2020-12-19,gap-men_279.jpg
279,gap,Men's Clothing,Adult Face Mask (3-Pack),"[7.5"" x 4"", Made of triple-layer cotton., Mixe...",$15.00,$5.00,"[true black, multi black green blue, navy blue...",[100% Cotton Printed Poplin.],https://www.gap.com/browse/product.do?pid=6688...,https://www2.assets-gap.com/webcontent/0018/90...,2020-12-19,gap-men_280.jpg
280,gap,Men's Clothing,Adult Face Mask (3-Pack),"[7.5"" x 4"", Made of triple-layer cotton., Mixe...",$15.00,$5.00,"[true black, multi black green blue, navy blue...",[100% Cotton Printed Poplin.],https://www.gap.com/browse/product.do?pid=6688...,https://www1.assets-gap.com/webcontent/0018/88...,2020-12-19,gap-men_281.jpg
281,gap,Men's Clothing,Adult Face Mask (3-Pack),"[7.5"" x 4"", Made of triple-layer cotton., Mixe...",$15.00,$5.00,"[true black, multi black green blue, navy blue...",[100% Cotton Printed Poplin.],https://www.gap.com/browse/product.do?pid=6688...,https://www1.assets-gap.com/webcontent/0018/87...,2020-12-19,gap-men_282.jpg


In [61]:
second_pass.to_csv("gap-men.csv", index=False, encoding = 'utf-8-sig')

In [62]:
import os
import requests as req

new_pd = pd.read_csv('gap-men.csv', encoding='utf-8-sig')

os.mkdir('gap-men')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('gap-men/'+ "gap-men_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1